In [1]:
import os
import cv2
import math
import numpy as np

In [2]:
def wordSegmentation(img, kernelSize=25, sigma=11, theta=7, minArea=0):

    kernel = createKernel(kernelSize, sigma, theta)
    imgFiltered = cv2.filter2D(img, -1, kernel, borderType=cv2.BORDER_REPLICATE).astype(np.uint8)
    (_, imgThres) = cv2.threshold(imgFiltered, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    imgThres = 255 - imgThres

    (components, _) = cv2.findContours(imgThres, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    res = []
    for c in components:

        if cv2.contourArea(c) < minArea:
            continue

        currBox = cv2.boundingRect(c) # returns (x, y, w, h)
        (x, y, w, h) = currBox
        currImg = img[y:y+h, x:x+w]
        res.append((currBox, currImg))

    return sorted(res, key=lambda entry:entry[0][0])

In [3]:
def prepareImg(img, height):

    assert img.ndim in (2, 3)
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h = img.shape[0]
    factor = height / h
    return cv2.resize(img, dsize=None, fx=factor, fy=factor)

In [4]:
def createKernel(kernelSize, sigma, theta):

    assert kernelSize % 2 
    halfSize = kernelSize // 2

    kernel = np.zeros([kernelSize, kernelSize])
    sigmaX = sigma
    sigmaY = sigma * theta

    for i in range(kernelSize):
        for j in range(kernelSize):
            x = i - halfSize
            y = j - halfSize

            expTerm = np.exp(-x**2 / (2 * sigmaX) - y**2 / (2 * sigmaY))
            xTerm = (x**2 - sigmaX**2) / (2 * math.pi * sigmaX**5 * sigmaY)
            yTerm = (y**2 - sigmaY**2) / (2 * math.pi * sigmaY**5 * sigmaX)

            kernel[i, j] = (xTerm + yTerm) * expTerm

    kernel = kernel / np.sum(kernel)
    return kernel

In [5]:
def main():

    imgFiles = os.listdir('data/')
    for (i,f) in enumerate(imgFiles):
    
        img = prepareImg(cv2.imread('data/%s'%f), 50)

        res = wordSegmentation(img, kernelSize=25, sigma=11, theta=7, minArea=100)
    
        if not os.path.exists('out/%s'%f):
            os.mkdir('out/%s'%f)

        print('Segmented into %d words'%len(res))
        for (j, w) in enumerate(res):
            (wordBox, wordImg) = w
            (x, y, w, h) = wordBox
            cv2.imwrite('out/%s/%d.png'%(f, j), wordImg)
            cv2.rectangle(img,(x,y),(x+w,y+h),0,1)

        cv2.imwrite('out/%s/summary.png'%f, img)

if __name__ == '__main__':
    main()


Segmented into 1 words
Segmented into 1 words
Segmented into 1 words
